<a href="https://colab.research.google.com/github/omararh/Expression-editor-assistant/blob/main/asisistant4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
print("La version de torch est : ", torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

La version de torch est :  2.6.0+cu124
Le calcul GPU est disponible ?  False


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer


MODEL_NAME = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,  # use float16
    device_map="auto",          # GPU if GPU else CPU
    trust_remote_code=True      # required for Phi-2
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# Import Prompt used data

In [4]:
AVAILABLE_CONTEXT = {
    "ContextVars": {
      "legacyContext.contextVar.urlBaseLicence": "URL de base de la licence",
      "legacyContext.contextVar.userId": "ID de l'utilisateur courant",
      "legacyContext.contextVar.indicatorId": "ID de l'indicateur",
      "legacyContext.contextVar.userName": "Nom de l'utilisateur",
      "legacyContext.contextVar.userUsername": "Nom d'utilisateur",
      "legacyContext.contextVar.userEmail": "Email de l'utilisateur",
      "legacyContext.contextVar.currentLanguage": "Langue courante",
      "legacyContext.contextVar.currentItemId": "ID de l'élément courant",
      "legacyContext.contextVar.currentCategoryId": "ID de la catégorie courante",
      "legacyContext.contextVar.userSession": "Session de l'utilisateur",
      "legacyContext.contextVar.basePath": "Chemin de base",
      "legacyContext.contextVar.workflowIsClosed": "État de fermeture du workflow (booléen)",
      "legacyContext.contextVar.userAccessLevels": "Niveaux d'accès de l'utilisateur (tableau)",
      "legacyContext.contextVar.groups": "Groupes de l'utilisateur (tableau)"
    },
    "ContextActions": {
      "commentShow()": "Afficher les commentaires",
      "commentDelete()": "Supprimer un commentaire",
      "commentDeleteAll()": "Supprimer tous les commentaires",
      "iconClick()": "Cliquer sur une icône",
      "crosstableConfig()": "Configurer une table croisée",
      "documentFileEdit()": "Éditer un fichier document",
      "documentAccessUpdateData()": "Mettre à jour les données d'un document",
      "documentAccessInvalidReports()": "Invalider les rapports d'un document",
      "documentCanConfigClosedWorkflow()": "Configurer un workflow fermé",
      "documentAddNewData()": "Ajouter de nouvelles données à un document",
      "documentEditData()": "Éditer les données d'un document",
      "documentMassUpdateData()": "Mettre à jour en masse les données d'un document",
      "documentDeleteData()": "Supprimer les données d'un document",
      "documentExport()": "Exporter un document",
      "documentImport()": "Importer un document",
      "documentAccessApi()": "Accéder à l'API du document"
    },
    "Fonctions": [
      "range(start, end, step)",
      "isEmpty(value)",
      "isNull(value)",
      "parseInt(value)",
      "if(condition, consequent, alternate)",
      "pregMatch(pattern, value)",
      "date(format, function)",
      "strtotime(format)",
      "mktime(hour, minute, second, month, day, year)",
      "replace(subject, search, replace)",
      "replaceAll(subject, search, replace)",
      "isString(str)",
      "toString(str)",
      "camelCase(str)",
      "capitalize(str)",
      "split(str, separator, limit)",
      "substrCompare(string1, string2, startPos, length, case)",
      "inString(substring, string)",
      "lowerCase(string)",
      "isArray(array)",
      "includes(haystack, needle)",
      "inArray(needle, haystack)",
      "concat(array1, array2, array3, array4, array5)",
      "reverse(array)",
      "slice(array, start, end)",
      "findIndex(array, value)",
      "inRange(number, start, end)",
      "subtract(minuend, subtrahend)",
      "isBoolean(bool)",
      "toBoolean(value)"
    ],
    "DateFunctions": {
      "firstDayOfMonth()": "Renvoie le premier jour du mois courant (format par défaut: 'Y-m-d')",
      "firstDayOfNextMonth()": "Renvoie le premier jour du mois suivant (format par défaut: 'Y-m-d')",
      "firstDayOfPreviousMonth()": "Renvoie le premier jour du mois précédent (format par défaut: 'Y-m-d')",
      "firstDayOfMonthMinusTwo()": "Renvoie le premier jour d'il y a deux mois (format par défaut: 'Y-m-d')",
      "firstDayOfMonthMinusThree()": "Renvoie le premier jour d'il y a trois mois (format par défaut: 'Y-m-d')",
      "lastDayOfPreviousMonth()": "Renvoie le dernier jour du mois précédent (format par défaut: 'Y-m-d')",
      "lastDayOfNextMonth()": "Renvoie le dernier jour du mois suivant (format par défaut: 'Y-m-d')",
      "lastDayOfMonth()": "Renvoie le dernier jour du mois courant (format par défaut: 'Y-m-d')",
      "lastSunday()": "Renvoie le dimanche de la semaine dernière (format par défaut: 'Y-m-d')",
      "monday()": "Renvoie le lundi de cette semaine (format par défaut: 'Y-m-d')",
      "nextMonday()": "Renvoie le lundi de la semaine prochaine (format par défaut: 'Y-m-d')",
      "nextSunday()": "Renvoie le dimanche de la semaine prochaine (format par défaut: 'Y-m-d')",
      "sunday()": "Renvoie le dimanche de cette semaine (format par défaut: 'Y-m-d')",
      "today()": "Renvoie la date du jour (format par défaut: 'Y-m-d')",
      "yesterday()": "Renvoie la date d'hier (format par défaut: 'Y-m-d')",
      "fiveDayAgo()": "Renvoie la date d'il y a 5 jours (format par défaut: 'Y-m-d')",
      "now()": "Renvoie la date et l'heure actuelles (format par défaut: 'Y-m-d H:i:s')",
      "yearOfPreviousMonth()": "Renvoie l'année du mois précédent (format par défaut: 'Y')",
      "yearOfMonthMinusTwo()": "Renvoie l'année d'il y a deux mois (format par défaut: 'Y')",
      "nextWeek()": "Renvoie le numéro de la semaine prochaine (format par défaut: 'W')",
      "previousWeek()": "Renvoie le numéro de la semaine précédente (format par défaut: 'W')",
      "previousMonth()": "Renvoie le numéro du mois précédent (format par défaut: 'm')",
      "week()": "Renvoie le numéro de la semaine courante (format par défaut: 'W')",
      "month()": "Renvoie le numéro du mois courant (format par défaut: 'm')",
      "year()": "Renvoie l'année courante (format par défaut: 'Y')"
    }
  }

SYMFONY_EXAMPLES = {
    "legacyContext.contextVar.userName == 'user_name'": "Tester si le nom d'utilisateur est égal à 'user_name'",
    "legacyContext.contextVar.userEmail matches '/@company\\.com$/'": "Vérifier si l'email de l'utilisateur se termine par @company.com",
    "legacyContext.contextVar.userId != null": "Vérifier si l'ID utilisateur est défini (non null)",
    "inArray('admin', legacyContext.contextVar.userAccessLevels)": "Vérifier si l'utilisateur a le niveau d'accès 'admin'",
    "legacyContext.contextVar.groups contains 'finance'": "Vérifier si l'utilisateur appartient au groupe 'finance'",
    "documentEditData() == true": "Vérifier si l'action d'édition de données est disponible",
    "documentExport() or documentImport()": "Vérifier si l'utilisateur peut exporter ou importer des documents",
    "not legacyContext.contextVar.workflowIsClosed": "Vérifier si le workflow n'est pas fermé",
    "legacyContext.contextVar.currentLanguage == 'fr'": "Vérifier si la langue courante est le français",
    "isEmpty(legacyContext.contextVar.userSession)": "Vérifier si la session utilisateur est vide",
    "isNull(legacyContext.contextVar.currentItemId)": "Vérifier si l'ID de l'élément courant est null",
    "legacyContext.contextVar.indicatorId > 100": "Vérifier si l'ID de l'indicateur est supérieur à 100",
    "legacyContext.contextVar.indicatorId >= 50 and legacyContext.contextVar.indicatorId <= 100": "Vérifier si l'ID de l'indicateur est entre 50 et 100 inclus",
    "inRange(parseInt(legacyContext.contextVar.indicatorId), 1, 10)": "Vérifier si l'ID de l'indicateur est entre 1 et 9",
    "documentCanConfigClosedWorkflow() and legacyContext.contextVar.workflowIsClosed": "Vérifier si l'utilisateur peut configurer un workflow qui est déjà fermé",
    "today() == '2025-04-15'": "Vérifier si la date du jour est le 15 avril 2025",
    "date('Y-m') == '2025-04'": "Vérifier si le mois courant est avril 2025",
    "inString('admin', legacyContext.contextVar.userEmail)": "Vérifier si l'email de l'utilisateur contient 'admin'",
    "pregMatch('/^[A-Z][a-z]+$/', legacyContext.contextVar.userName)": "Vérifier si le nom d'utilisateur commence par une majuscule suivie de minuscules",
    "legacyContext.contextVar.userAccessLevels contains 'admin' or legacyContext.contextVar.userAccessLevels contains 'manager'": "Vérifier si l'utilisateur est un admin ou un manager",
    "if(documentEditData(), 'Peut éditer', 'Ne peut pas éditer')": "Retourner un message selon les droits d'édition",
    "if(isEmpty(legacyContext.contextVar.userName), 'Anonyme', legacyContext.contextVar.userName)": "Utiliser 'Anonyme' si le nom d'utilisateur est vide, sinon utiliser le nom",
    "capitalize(legacyContext.contextVar.userName) == 'user_name'": "Vérifier si le nom d'utilisateur avec première lettre majuscule est 'user_name'",
    "lowerCase(legacyContext.contextVar.currentLanguage) == 'fr'": "Vérifier si la langue courante en minuscules est 'fr'",
    "isString(legacyContext.contextVar.userId) and parseInt(legacyContext.contextVar.userId) > 0": "Vérifier si l'ID utilisateur est une chaîne et peut être converti en entier positif",
    "legacyContext.contextVar.groups starts with 'admin'": "Vérifier si le groupe commence par 'admin'",
    "legacyContext.contextVar.userEmail ends with '@company.com'": "Vérifier si l'email se termine par '@company.com'",
    "replaceAll(legacyContext.contextVar.userUsername, ' ', '') == 'johnsmith'": "Vérifier si le nom d'utilisateur sans espaces est 'johnsmith'",
    "legacyContext.contextVar.userId === 42": "Vérifier si l'ID utilisateur est strictement égal à 42 (même type)",
    "isArray(legacyContext.contextVar.userAccessLevels) and not isEmpty(legacyContext.contextVar.userAccessLevels)": "Vérifier si les niveaux d'accès sont un tableau non vide",
    "week() == previousWeek() + 1": "Vérifier si la semaine actuelle suit directement la semaine précédente",
    "month() == previousMonth() + 1 or (month() == 1 and previousMonth() == 12)": "Vérifier si le mois actuel suit directement le mois précédent (avec gestion du changement d'année)",
    "concat(['admin'], legacyContext.contextVar.userAccessLevels) contains 'admin'": "Ajouter 'admin' aux niveaux d'accès et vérifier si 'admin' est présent",
    "legacyContext.contextVar.indicatorId % 2 == 0": "Vérifier si l'ID de l'indicateur est pair",
    "legacyContext.contextVar.userEmail matches '/^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$/'": "Vérifier si l'email a un format valide",
    "strtotime('+1 week') > now()": "Vérifier si la date dans une semaine est dans le futur",
    "(legacyContext.contextVar.userAccessLevels contains 'admin') ? true : documentEditData()": "Autoriser si admin, sinon vérifier les droits d'édition",
    "subtract(year(), yearOfPreviousMonth()) <= 1": "Vérifier si l'année du mois précédent est la même que l'année actuelle ou une année avant",
    "split(legacyContext.contextVar.userName, ' ')[0] == 'John'": "Vérifier si le prénom est 'John'",
    "findIndex(legacyContext.contextVar.groups, 'admin') >= 0": "Vérifier si 'admin' existe dans les groupes",
    "isBoolean(legacyContext.contextVar.workflowIsClosed) and legacyContext.contextVar.workflowIsClosed": "Vérifier si l'état de fermeture du workflow est un booléen et est vrai",
    "toBoolean(legacyContext.contextVar.workflowIsClosed)": "Convertir l'état de fermeture du workflow en booléen",
    "commentShow() == true": "Vérifier si l'affichage des commentaires est activé",
    "documentFileEdit() and not documentDeleteData()": "Vérifier si l'utilisateur peut éditer des fichiers mais pas supprimer de données",
    "documentAccessApi() == false": "Vérifier si l'accès à l'API du document est désactivé",
    "commentDelete() or commentDeleteAll()": "Vérifier si l'utilisateur peut supprimer des commentaires",
    "getModelValue('IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED')": "Exécuter le modèle 'IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED'",
    "getModelValue('IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED') || false": "Exécuter le modèle 'IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED' et si le résultat est undefined ou null, retourner false",
    "getModelValue('IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED','file_cat_52') || false": "Exécuter le modèle 'IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED' et si le résultat pour 'file_cat_52' est undefined ou null, retourner false",
    "getModelValue('IPC_MIG_STATUS_GROUP_MEMBER_TO_CREATE','action_status')": "Exécuter le modèle 'IPC_MIG_STATUS_GROUP_MEMBER_TO_CREATE' et récupérer la valeur pour 'action_status'",
    "getModelValue('IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED','file_cat_140')": "Exécuter le modèle 'IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED' et récupérer la valeur pour 'file_cat_140'",
    "getModelValue('IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED','file_cat_92')": "Exécuter le modèle 'IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED' et récupérer la valeur pour 'file_cat_92'",
    "getModelValue('IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED','file_cat_146') || false": "Exécuter le modèle 'IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED' et si le résultat pour 'file_cat_146' est undefined ou null, retourner false",
    "getModelValue('IPC_GOV_OWNER_LIST_CONFIG','id')": "Exécuter le modèle 'IPC_GOV_OWNER_LIST_CONFIG' et récupérer la valeur pour 'id'",
    "getModelValue('IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED','file_cat_69') || false": "Exécuter le modèle 'IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED' et si le résultat pour 'file_cat_69' est undefined ou null, retourner false",
    "getModelValue('IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED','file_cat_105')": "Exécuter le modèle 'IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED' et récupérer la valeur pour 'file_cat_105'",
    "getModelValue('IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED','file_cat_127')": "Exécuter le modèle 'IPC_GMD_PROV_ACCOUNT_FILE_CAT_REQUIRED' et récupérer la valeur pour 'file_cat_127'",
    "row.rule_order ~ \"/\" ~row.input_field~\"/\"~row.tested_value": "Pour chaque ligne du model du document, concaténer les valeurs des colonnes 'rule_order', 'input_field' et 'tested_value' avec des séparateurs '/'",
    "row.request_owner": "Pour chaque ligne du model ou du document, récupérer la valeur de la colonne 'request_owner'",
    "row.request_owner == 'user_name'": "Pour chaque ligne du model ou du document, vérifier que la valeur de la colonne 'request_owner' est égale a user_name",
    "row.rule_order ~ \"/\" ~row.input_field~\"/\"~row.tested_value": "Pour chaque ligne du model ou du document, concaténer les valeurs des colonnes 'rule_order', 'input_field' et 'tested_value' avec des séparateurs '/'",
    "row.virtual_1684507387": "Pour chaque ligne du model ou du document, récupérer la valeur de la colonne 'virtual_1684507387'"
}

# Embedding to filter the prompt data



In [5]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download fr_core_news_md

# Load french tokenizer, tagger, parser
nlp = spacy.load('fr_core_news_md')

def get_embedding(text):
    """
    get text embedding
    """
    doc = nlp(text)
    if doc.vector.any():
        return doc.vector / np.linalg.norm(doc.vector)
    return doc.vector


def select_relevant_examples(user_query, examples_dict, top_k=5):
    """
    Sélectionne les exemples les plus pertinents en fonction de la similarité
    d'embedding avec la requête utilisateur.

    Args:
        user_query: La requête de l'utilisateur
        examples_dict: Dictionnaire d'exemples (description -> expression)
        top_k: Nombre d'exemples à retourner

    Returns:
        Liste des top_k exemples les plus pertinents (tuples de description, expression)
    """
    query_embedding = get_embedding(user_query)

    similarities = []

    for description, expression in examples_dict.items():
        desc_embedding = get_embedding(description)

        similarity = cosine_similarity([query_embedding], [desc_embedding])[0][0]
        similarities.append((similarity, description, expression))

    similarities.sort(reverse=True)
    top_examples = [(desc, expr) for _, desc, expr in similarities[:top_k]]

    return top_examples


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 12.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


 # Prompt Functions

In [6]:
def generate_expression(user_intent, available_context=None, examples=None, max_retries=1):
    """
    Génère une suggestion d'expression Symfony avec retry en cas d'échec
    Args:
        user_intent (str): Intention ou question de l'utilisateur
        available_context (dict, optional): Contexte disponible dans l'éditeur
        examples (list, optional): Exemples d'expressions valides
        max_retries (int, optional): Nombre maximal de tentatives en cas d'erreur
    Returns:
        str: Expression Symfony suggérée par le modèle
    """
    # Construction du prompt pour Phi-2
    prompt = _build_prompt(user_intent, available_context, examples)

    retries = 0
    while retries <= max_retries:
        try:
            # Génération et traitement de la réponse
            response = _generate_and_process_response(prompt)

            # Vérification que la réponse n'est pas vide
            if _is_valid_response(response):
                return response
            else:
                # Si réponse vide ou invalide, retenter
                retries = _handle_retry(retries, max_retries, "Réponse vide ou invalide")
        except Exception as e:
            # En cas d'erreur, retenter
            retries = _handle_retry(retries, max_retries, f"Erreur lors de la génération: {e}")

    # Si après toutes les tentatives, rien n'a fonctionné
    return "Une erreur est survenue, veuillez réessayer."


def _build_prompt(user_intent, available_context, examples):
    """Construit le prompt pour le modèle"""
    prompt = "Tu es un assistant spécialisé pour l'éditeur d'expression Symfony (https://symfony.com/doc/current/reference/formats/expression_language.html).\n"
    prompt += "Tu aides les consultants à construire des expressions conditionnelles valides en utilisant la syntaxe Symfony Expression Language.\n\n"

    # Ajout du contexte disponible
    if available_context:
        prompt += "Éléments disponibles dans l'environnement:\n"
        for category, items in available_context.items():
            prompt += f"- {category}: {', '.join(items)}\n"

    # Ajout d'exemples
    if examples:
        prompt += "\n### EXEMPLES PERTINENTS ###\n"
        # Inverser le dictionnaire car dans SYMFONY_EXAMPLES, les expressions sont les clés et les descriptions sont les valeurs
        inverted_examples = {value: key for key, value in examples.items()}
        relevant_examples = select_relevant_examples(user_intent, inverted_examples, top_k=5)
        print("relevant_examples ==> ", relevant_examples)
        for description, expression in relevant_examples:
            prompt += f"Description: {description}\n"
            prompt += f"Expression: {expression}\n\n"

    # Ajouter des instructions spécifiques
    prompt += "\nRègles à suivre:\n"
    prompt += "- Utilise uniquement la syntaxe de Symfony Expression Language\n"
    prompt += "- N'utilise que les variables, contextes et méthodes listés ci-dessus\n"

    # Ajouter la demande de l'utilisateur
    prompt += f"Demande: {user_intent}\n\n"
    prompt += "Expression Symfony: "

    return prompt


def _generate_and_process_response(prompt):
    """Génère une réponse et extrait la partie pertinente"""
    # Encoder le prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Génération de la réponse
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.5,
        do_sample=True,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id
    )

    # Décodage de la sortie
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraction de la partie pertinente
    return _extract_response(full_response, prompt)


def _extract_response(full_response, prompt):
    """Extrait et nettoie la partie pertinente de la réponse"""
    # Extraction après "Expression Symfony: "
    if "Expression Symfony: " in full_response:
        response = full_response.split("Expression Symfony: ")[-1].strip()
    else:
        # Fallback au cas où le modèle ne respecte pas le format
        response = full_response.replace(prompt, "").strip()

    # Nettoyage final (retirer les explications qui pourraient suivre l'expression)
    if "\n" in response:
        response = response.split("\n")[0].strip()

    return response


def _is_valid_response(response):
    """Vérifie si la réponse est valide"""
    return response and not response.isspace() and "```" not in response


def _handle_retry(retries, max_retries, error_message):
    """Gère la logique de retry et renvoie le nouveau compteur"""
    retries += 1
    print(f"{error_message}, tentative {retries}/{max_retries+1}...")

    # Attendre un peu avant de retenter
    import time
    time.sleep(1)

    return retries

# Prompts

In [ ]:
user_query_2 = "Déterminer si l’utilisateur peut effectuer l’export ou l’import de documents"
result_2 = generate_expression(user_query_2, AVAILABLE_CONTEXT, SYMFONY_EXAMPLES)
print("Résultat:")
print(result_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


relevant_examples ==>  [("Vérifier si l'utilisateur peut exporter ou importer des documents", 'documentExport() or documentImport()'), ("Vérifier si l'accès à l'API du document est désactivé", 'documentAccessApi() == false'), ("Vérifier si l'utilisateur peut supprimer des commentaires", 'commentDelete() or commentDeleteAll()'), ("Vérifier si l'action d'édition de données est disponible", 'documentEditData() == true'), ("Vérifier si l'ID de l'indicateur est supérieur à 100", 'legacyContext.contextVar.indicatorId > 100')]
